In [1]:
pip install bs4

     |████████████████████████████████| 115 kB 3.4 MB/s eta 0:00:01
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=ddc1ac5062c11d41a2ab953de8a8ed1cf3cd58b793fe62ab5ecb002da5f0dd5a
  Stored in directory: /home/jovyan/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


### Load data

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

from bs4 import BeautifulSoup 
import requests

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = requests.get(url).text

soup = BeautifulSoup(html, 'html.parser')
table = soup.find('table')
rows = table.tbody.find_all("tr")

column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

for row in rows:
    data=[]
    for td in row.find_all('td'):
        data.append(td.text.strip())
    if len(data)==3:
        df.loc[len(df)] = data

df=df[df['Borough']!='Not assigned']

df["Neighborhood"] = df["Neighborhood"].str.replace("\n","")
df = df.groupby(["Postalcode", "Borough"])["Neighborhood"].apply(", ".join).reset_index()
df

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


### Get latitude and longitude

In [4]:
geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_mrg = pd.merge(df, geo, how='left', left_on = 'Postalcode', right_on = 'Postal Code')
df_mrg.drop("Postal Code", axis=1, inplace=True)
#df_mrg.drop(["Postal Code","Latitude_y", "Longitude_y"], axis=1, inplace=True)
#df_mrg.rename(columns={'Latitude_x': 'Latitude', 'Longitude_x': 'Longitude'}, inplace=True)
df_mrg.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Explore and cluster the neighborhoods in Toronto

In [5]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /srv/conda/envs/notebook

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0

The fol

In [6]:
pip install folium

     |████████████████████████████████| 93 kB 1.2 MB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [7]:
import folium

In [8]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


#### Use geopy library to get the latitude and longitude values of Toronto

In [9]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [10]:
neighborhoods = df_mrg
neighborhoods.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [11]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [12]:
# get the geographical coordinates of Scarborough
scarborough_data = neighborhoods[neighborhoods['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [13]:
address = 'Scarborough, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [14]:
# create map of Scarborough using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

#### Define Foursquare Credentials and Version

In [15]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SAYKPMTVW4I2Q2UWB2GDGKVGAZBTLXZRKHPWT020MN4JUMAB
CLIENT_SECRET:OQMSY1SEBOB5YJRDPNMHP4UF0QWAGPNPYCLFZ15QV2OQ50NB


In [29]:
# explore the first neighborhood in our dataframe
scarborough_data.loc[[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16], 'Neighborhood']

0                                        Malvern, Rouge
1                Rouge Hill, Port Union, Highland Creek
2                     Guildwood, Morningside, West Hill
3                                                Woburn
4                                             Cedarbrae
5                                   Scarborough Village
6           Kennedy Park, Ionview, East Birchmount Park
7                       Golden Mile, Clairlea, Oakridge
8       Cliffside, Cliffcrest, Scarborough Village West
9                           Birch Cliff, Cliffside West
10    Dorset Park, Wexford Heights, Scarborough Town...
11                                    Wexford, Maryvale
12                                            Agincourt
13              Clarks Corners, Tam O'Shanter, Sullivan
14    Milliken, Agincourt North, Steeles East, L'Amo...
15                        Steeles West, L'Amoreaux West
16                                          Upper Rouge
Name: Neighborhood, dtype: object

In [30]:
# Get the neighborhood's latitude and longitude values.
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern, Rouge are 43.806686299999996, -79.19435340000001.


In [31]:
# create the GET request URL
LIMIT = 100
radius = 1000

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=SAYKPMTVW4I2Q2UWB2GDGKVGAZBTLXZRKHPWT020MN4JUMAB&client_secret=OQMSY1SEBOB5YJRDPNMHP4UF0QWAGPNPYCLFZ15QV2OQ50NB&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=1000&limit=100'

In [32]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f3d8a61b52da7221bb12d4d'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 16,
  'suggestedBounds': {'ne': {'lat': 43.81568630900001,
    'lng': -79.18190576146081},
   'sw': {'lat': 43.797686290999984, 'lng': -79.20680103853921}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d669cba83865481c948fa53',
       'name': 'Images Salon & Spa',
       'location': {'address': '8130 Sheppard Ave E',
        'crossStreet': 'Morningside Ave',
        'lat': 43.80228301948931,
        'lng': -79.19856472801668,
        'labeledLatLngs'

In [33]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [35]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Images Salon & Spa,Spa,43.802283,-79.198565
1,Harvey's,Restaurant,43.800020,-79.198307
2,Wendy’s,Fast Food Restaurant,43.807448,-79.199056
3,Wendy's,Fast Food Restaurant,43.802008,-79.198080
4,RBC Royal Bank,Bank,43.798782,-79.197090
5,Caribbean Wave,Caribbean Restaurant,43.798558,-79.195777
6,Staples Morningside,Paper / Office Supplies Store,43.800285,-79.196607
7,Tim Hortons,Coffee Shop,43.802000,-79.198169
8,Lee Valley,Hobby Shop,43.803161,-79.199681
9,Tim Hortons / Esso,Coffee Shop,43.801863,-79.199296


In [36]:
# venues were returned by Foursquare
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

16 venues were returned by Foursquare.


#### Explore Neighborhoods in Scarborough

In [37]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [38]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge


In [39]:
# check the size of the dataframe
print(scarborough_venues.shape)
scarborough_venues

(89, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant
5,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
6,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
7,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center
8,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Lawrence Ave E & Kingston Rd,43.767704,-79.189490,Intersection
9,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Eggsmart,43.767800,-79.190466,Breakfast Spot


In [40]:
# venues were returned for each neighborhood
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,9,9,9,9,9,9
"Clarks Corners, Tam O'Shanter, Sullivan",12,12,12,12,12,12
"Cliffside, Cliffcrest, Scarborough Village West",2,2,2,2,2,2
"Dorset Park, Wexford Heights, Scarborough Town Centre",7,7,7,7,7,7
"Golden Mile, Clairlea, Oakridge",10,10,10,10,10,10
"Guildwood, Morningside, West Hill",8,8,8,8,8,8
"Kennedy Park, Ionview, East Birchmount Park",6,6,6,6,6,6


In [41]:
# how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 54 uniques categories.


#### Analyze Each Neighborhood

In [42]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,American Restaurant,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,General Entertainment,Grocery Store,Gym,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [43]:
scarborough_onehot.shape

(89, 55)

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [44]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,General Entertainment,Grocery Store,Gym,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.250000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.25,0.000000,0.250000,0.000,0.0,0.000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.25,0.00,0.0,0.000000,0.000000
1,"Birch Cliff, Cliffside West",0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.25,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.0,0.000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.25,0.00,0.0,0.000000,0.000000
2,Cedarbrae,0.0,0.000000,0.111111,0.111111,0.111111,0.0,0.000000,0.0,0.000000,0.00,0.111111,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.111111,0.000000,0.111111,0.00,0.000000,0.000000,0.111111,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.111111,0.000,0.0,0.000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.00,0.00,0.0,0.111111,0.000000
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.0,0.000000,0.000000,0.000000,0.083333,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.083333,0.000000,0.00,0.083333,0.000000,0.000000,0.000,0.083333,0.083333,0.000000,0.083333,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.083333,0.000000,0.00,0.000000,0.000000,0.000,0.0,0.000,0.00,0.0,0.083333,0.000000,0.000000,0.083333,0.166667,0.000000,0.000,0.000,0.000000,0.00,0.00,0.0,0.083333,0.000000
4,"Cliffside, Cliffcrest, Scarborough Village West",0.5,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.0,0.000,0.00,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.00,0.00,0.0,0.000000,0.000000
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.142857,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.142857,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.285714,0.000,0.000000,0.000000,0.00,0.142857,0.000000,0.000,0.0,0.000,0.00,0.0,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.00,0.00,0.0,0.000000,0.142857
6,"Golden Mile, Clairlea, Oakridge",0.0,0.000000,0.000000,0.200000,0.000000,0.0,0.000000,0.2,0.100000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.1,0.000000,0.100,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.1,0.000,0.00,0.0,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.00,0.00,0.1,0.000000,0.000000
7,"Guildwood, Morningside, West Hill",0.0,0.000000,0.000000,0.000000,0.125000,0.0,0.125000,0.0,0.000000,0.00,0.000000,0.

In [45]:
scarborough_grouped.shape

(16, 55)

In [46]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [55]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,College Stadium,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint
1,"Birch Cliff, Cliffside West",College Stadium,General Entertainment,Skating Rink,Café,Vietnamese Restaurant,Grocery Store,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
2,Cedarbrae,Thai Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Lounge,Hakka Restaurant,Discount Store
3,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Fast Food Restaurant,Italian Restaurant,Chinese Restaurant,Noodle House,Pharmacy,Convenience Store,Fried Chicken Joint,Bank,Gas Station
4,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Motel,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


#### Clustering neighbourhoods

In [ ]:
# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 3

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scarborough_merged = scarborough_data

# merge scarborough_grouped with scarborough_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scarborough_merged.head() # check the last columns!

In [58]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
scarborough_nonan = scarborough_merged.dropna(subset=['Cluster Labels'])

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_nonan['Latitude'], scarborough_nonan['Longitude'], scarborough_nonan['Neighborhood'], scarborough_nonan['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examine clusters

In [56]:
# cluster 1
scarborough_nonan.loc[scarborough_nonan['Cluster Labels'] == 0, scarborough_nonan.columns[[1] + list(range(5, scarborough_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,0.0,Spa,Playground,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant
14,Scarborough,0.0,Playground,Park,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant


In [57]:
# cluster 2
scarborough_nonan.loc[scarborough_nonan['Cluster Labels'] == 1, scarborough_nonan.columns[[1] + list(range(5, scarborough_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,1.0,Intersection,Bank,Restaurant,Rental Car Location,Breakfast Spot,Electronics Store,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Convenience Store
3,Scarborough,1.0,Coffee Shop,Soccer Field,Korean Restaurant,Indian Restaurant,Auto Garage,Department Store,Grocery Store,General Entertainment,Athletics & Sports,Gas Station
4,Scarborough,1.0,Thai Restaurant,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Gas Station,Caribbean Restaurant,Lounge,Fried Chicken Joint,Discount Store
6,Scarborough,1.0,Discount Store,Chinese Restaurant,Coffee Shop,Department Store,Vietnamese Restaurant,Convenience Store,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station
7,Scarborough,1.0,Bakery,Ice Cream Shop,Soccer Field,Intersection,Bus Station,Metro Station,Department Store,Grocery Store,General Entertainment,Gas Station
9,Scarborough,1.0,College Stadium,General Entertainment,Skating Rink,Café,Vietnamese Restaurant,Grocery Store,Gas Station,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant
10,Scarborough,1.0,Indian Restaurant,Pet Store,Chinese Restaurant,Gaming Cafe,Light Rail Station,Vietnamese Restaurant,Restaurant,Coffee Shop,Gas Station,Soccer Field
11,Scarborough,1.0,Middle Eastern Restaurant,Auto Garage,Bakery,Sandwich Place,Breakfast Spot,Vietnamese Restaurant,Grocery Store,General Entertainment,Gas Station,Gaming Cafe
12,Scarborough,1.0,Skating Rink,Latin American Restaurant,Breakfast Spot,Lounge,Vietnamese Restaurant,Convenience Store,General Entertainment,Gas Station,Gaming Cafe,Fried Chicken Joint
13,Scarborough,1.0,Pizza Place,Convenience Store,Thai Restaurant,Italian Restaurant,Bank,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Chinese Restaurant,Noodle House


In [58]:
# cluster 3
scarborough_nonan.loc[scarborough_nonan['Cluster Labels'] == 2, scarborough_nonan.columns[[1] + list(range(5, scarborough_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,2.0,Fast Food Restaurant,Vietnamese Restaurant,Thai Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Gaming Cafe,Fried Chicken Joint,Electronics Store


In [59]:
# cluster 4
scarborough_nonan.loc[scarborough_nonan['Cluster Labels'] == 3, scarborough_nonan.columns[[1] + list(range(5, scarborough_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Scarborough,3.0,American Restaurant,Intersection,Motel,Convenience Store,Grocery Store,General Entertainment,Gas Station,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant


In [60]:
# cluster 5
scarborough_nonan.loc[scarborough_nonan['Cluster Labels'] == 4, scarborough_nonan.columns[[1] + list(range(5, scarborough_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,4.0,Moving Target,Bar,Vietnamese Restaurant,Grocery Store,General Entertainment,Gas Station,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
